<a href="https://colab.research.google.com/github/VieuxRoblochon/Deep-Image-Prior/blob/master/02_Correct_attenuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Correct reflectivity attenuation

In this example the reflectivity attenuation is calculated and then corrected
for a polarimetric radar using a Z-PHI method implemented in Py-ART.


In [ ]:
pip install arm-pyart

In [13]:
#print(__doc__)

# Author: Jonathan J. Helmus (jhelmus@anl.gov)
# License: BSD 3 clause

import matplotlib.pyplot as plt

import pyart
import os
import glob as glob
import datetime as datetime

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
from google.colab import drive
drive.mount("/ShardDrives/")

Drive already mounted at /ShardDrives/; to attempt to forcibly remount, call drive.mount("/ShardDrives/", force_remount=True).


In [ ]:
folderpath = f'/ShardDrives/MyDrive/Colab Notebooks (1)/USBR_SWE/radar_data_KGJX/KGJX20230322' 
os.listdir(folderpath)

In [ ]:
folderpath = f'/ShardDrives/MyDrive/Colab Notebooks (1)/USBR_SWE/radar_data_KGJX/KGJX20230322' 
files      = os.listdir(folderpath)
# Load Level II data file
radarsite = 'KGJX'
print(f"{len(files)} files in directory {folderpath}")

file_pattern = os.path.join(folderpath, '*')
files = glob.glob(file_pattern)
radars = []
for file in files:
  # Read Level II files using pyart
  radar = pyart.io.read_nexrad_archive(file)
  radars.append(radar) # Add radar object to the list
      # Call the function to plot the figure
      #plot_fig(radar, 'SDUS65_N0Q', 0)
print(radar.info())
# Retrieve dual-polarization moments
reflectivity_h = radar.fields['reflectivity_h']['data']
reflectivity_v = radar.fields['reflectivity_v']['data']
differential_reflectivity = radar.fields['differential_reflectivity']['data']
correlation_coefficient = radar.fields['correlation_coefficient']['data']
differential_phase = radar.fields['differential_phase']['data']
specific_differential_phase = radar.fields['specific_differential_phase']['data']

# Process or visualize the dual-polarization moments as needed
# For example, you can plot the reflectivity data:
display = pyart.graph.RadarDisplay(radar)
display.plot('reflectivity_h', title='Reflectivity (Horizontal)')


206 files in directory /ShardDrives/MyDrive/Colab Notebooks (1)/USBR_SWE/radar_data_KGJX/KGJX20230322


In [ ]:
#file = pyart.testing.get_test_data("sgpcsaprsurcmacI7.c0.20110520.095101.nc")

def corr_radar(radar):
    # perform attenuation correction
    spec_at, cor_z = pyart.correct.calculate_attenuation(
        radar,
        0,
        refl_field="reflectivity",
        ncp_field=None,
        rhv_field="cross_correlation_ratio",
        phidp_field="specific_differential_phase",
    )
    radar.add_field("specific_attenuation", spec_at)
    radar.add_field("corrected_reflectivity_horizontal", cor_z)

    # create the plot
    fig = plt.figure(figsize=(15, 5))
    ax1 = fig.add_subplot(131)
    display = pyart.graph.RadarDisplay(radar)
    display.plot(
        "reflectivity_horizontal",
        0,
        ax=ax1,
        vmin=0,
        vmax=60.0,
        colorbar_label="",
        title="Raw Reflectivity",
    )

    ax2 = fig.add_subplot(132)
    display.plot(
        "specific_attenuation",
        0,
        vmin=0,
        vmax=1.0,
        colorbar_label="",
        ax=ax2,
        title="Specific Attenuation",
    )

    ax3 = fig.add_subplot(133)
    display = pyart.graph.RadarDisplay(radar)
    display.plot(
        "corrected_reflectivity_horizontal",
        0,
        vmin=0,
        vmax=60.0,
        colorbar_label="",
        ax=ax3,
        title="Corrected Reflectivity",
    )

    plt.suptitle("Attenuation correction using Py-ART", fontsize=16)
    plt.show()

In [ ]:
prods = ['SDUS65_N1Q', 'SDUS85_N1K', 'SDUS85_N1X', 'SDUS85_N1C', 'SDUS75_N1A']

field_dict = {'SDUS65_N1Q': 'reflectivity',
              'SDUS85_N1K': 'differential_phase',
              'SDUS85_N1X': 'differential_reflectivity',
              'SDUS85_N1C': 'cross_correlation_ratio',
              'SDUS75_N1A': 'specific_attenuation'
             }

# Define area
bbox = [-107, 38.4367, -108.5, 39.5]

for prod in prods:
    folder_path = r'C:\GIS-RadarWorkspace\03182023_to_03252023'     #r'C:\GIS-RadarWorkspace\Radar_Data'
    fpattern = f'KGJT_{prod}GJX_20230322*' #KGJT_SDUS85_N0XGJX_20230322
    files = glob.glob(os.path.join(folder_path, fpattern))
    for file in files:
        radar = pyart.io.read_nexrad_level3(filename=file, field_name=prod)
        #print(radar.info())
        corr_radar(radar)
        #plot_fig(radar, prod, bbox)

KeyError: 'specific_differential_phase'

In [2]:
# Period list with non-overlapping date ranges
"""period_list = [['2021-02-02', '2021-02-04', 50.8, 0.1, 0.15],
 ['2021-02-05', '2021-02-09', 43.18, -4.94, 0.10],
 ['2021-02-13', '2021-02-17', 35.56, -2.14, 0.08],
 ['2021-02-18', '2021-02-22', 45.72, -6.66, 0.07],
 ['2021-02-23', '2021-02-27', 27.94, -8.67, 0.06],
 ['2021-02-28', '2021-03-04', 45.72, -2.85, 0.08],
 ['2021-03-05', '2021-03-09', 73.66, -5.96, 0.19],
 ['2021-03-10', '2021-03-14', 111.76, -3.08, 0.18],
 ['2021-03-15', '2021-03-19', 60.96, -0.47, 0.13],
 ['2021-03-20', '2021-03-24', 66.04, -3.15, 0.14],
 ['2021-03-25', '2021-03-29', 55.88, -3.51, 0.12],
 ['2021-03-30', '2021-04-02', 68.58, -0.73, 0.16],
 ['2021-04-03', '2021-04-07', 363.22, -2.01, 0.3],
 ['2021-04-08', '2021-04-12', 40.64, -5.06, 0.05],
 ['2021-04-13', '2021-04-17', 111.76, -0.03, 0.28],
 ['2021-04-18', '2021-04-22', 111.76, -0.03, 0.28],
 ['2021-04-23', '2021-04-27', 147.32, 2.65, 0.39],
 ['2021-04-28', '2021-04-30', 137.16, 3.23, 0.21],
 ['2021-10-01', '2021-10-04', 43.18, -0.16, 0.06],
 ['2021-10-05', '2021-10-09', 43.18, -0.16, 0.06],
 ['2021-10-10', '2021-10-14', 43.18, -0.16, 0.06],
 ['2021-10-15', '2021-10-19', 43.18, -0.16, 0.06],
 ['2021-10-20', '2021-10-24', 33.02, 2.22, 0.08],
 ['2021-10-25', '2021-10-29', 38.1, 3.38, 0.03],
 ['2021-10-30', '2021-11-03', 35.56, 1.84, 0.04],
 ['2021-11-04', '2021-11-08', 33.02, 3.08, 0.04],
 ['2021-11-09', '2021-11-13', 40.64, 2.71, 0.05],
 ['2021-11-14', '2021-11-18', 35.56, 3.56, 0.03],
 ['2021-11-19', '2021-11-23', 50.8, 2.49, 0.04],
 ['2021-11-24', '2021-11-28', 30.48, 0.93, 0.03],
 ['2021-11-29', '2021-12-02', 38.1, -0.16, 0.03],
 ['2021-12-03', '2021-12-07', 38.1, -1.04, 0.05],
 ['2021-12-08', '2021-12-12', 55.88, -4.7, 0.07],
 ['2021-12-13', '2021-12-17', 48.26, -4.18, 0.09],
 ['2021-12-18', '2021-12-22', 30.48, -3.73, 0.05],
 ['2021-12-23', '2021-12-27', 86.36, -0.68, 0.09],
 ['2021-12-28', '2021-12-31', 96.52, -6.97, 0.14],
 ['2022-01-01', '2022-01-04', 33.02, -3.96, 0.08],
 ['2022-01-05', '2022-01-09', 25.4, -2.37, 0.05],
 ['2022-01-10', '2022-01-14', 40.64, 1.37, 0.06],
 ['2022-01-30', '2022-02-02', 30.48, -4.38, 0.05],
 ['2022-01-15', '2022-01-19', 78.74, -0.78, 0.12],
 ['2022-01-20', '2022-01-24', 78.74, -4.55, 0.19],
 ['2022-01-25', '2022-01-29', 35.56, -4.38, 0.06],
 ['2022-02-03', '2022-02-07', 43.18, -5.0, 0.05],
 ['2022-02-08', '2022-02-11', 30.48, -7.33, 0.05],
 ['2022-02-12', '2022-02-15', 43.18, -1.13, 0.09],
 ['2022-02-16', '2022-02-20', 60.96, -1.76, 0.09],
 ['2022-02-21', '2022-02-25', 63.5, -4.5, 0.11],
 ['2022-02-26', '2022-02-28', 93.98, -3.4, 0.11],
 ['2022-03-01', '2022-03-05', 109.22, 2.54, 0.19],
 ['2022-03-06', '2022-03-10', 81.28, -3.16, 0.09],
 ['2022-03-11', '2022-03-15', 33.02, -8.43, 0.05],
 ['2022-03-16', '2022-03-20', 91.44, -2.49, 0.2],
 ['2022-03-21', '2022-03-25', 116.84, -1.58, 0.22],
 ['2022-03-26', '2022-03-31', 58.42, -2.5, 0.1],
 ['2022-04-01', '2022-04-05', 68.58, 3.07, 0.12],
 ['2022-04-06', '2022-04-11', 83.82, 1.32, 0.14],
 ['2022-04-12', '2022-04-16', 66.04, 0.99, 0.1],
 ['2022-04-17', '2022-04-21', 96.52, 2.89, 0.24],
 ['2022-04-22', '2022-04-26', 63.5, -3.27, 0.12],
 ['2022-04-27', '2022-04-30', 22.86, 2.67, 0.1]]"""
period_list = [['2022-05-01', '2022-05-04', 190.5, 6.3, 0.4],
 ['2022-05-05', '2022-05-09', 96.52, 4.58, 0.29],
 ['2022-05-10', '2022-05-14', 22.86, 7.82, 0.4],
 ['2022-05-15', '2022-05-19', 22.86, 7.82, 0.4],
 ['2022-05-20', '2022-05-24', 22.86, 7.82, 0.4],
 ['2022-05-25', '2022-05-29', 22.86, 7.82, 0.4],
 ['2022-05-30', '2022-06-01', 22.86, 7.82, 0.4]]
 
print(period_list)

[['2022-05-01', '2022-05-04', 190.5, 6.3, 0.4], ['2022-05-05', '2022-05-09', 96.52, 4.58, 0.29], ['2022-05-10', '2022-05-14', 22.86, 7.82, 0.4], ['2022-05-15', '2022-05-19', 22.86, 7.82, 0.4], ['2022-05-20', '2022-05-24', 22.86, 7.82, 0.4], ['2022-05-25', '2022-05-29', 22.86, 7.82, 0.4], ['2022-05-30', '2022-06-01', 22.86, 7.82, 0.4]]


In [3]:
def correct_dual_pol_moments(radar):
    # Perform attenuation correction
    spec_at, cor_z = pyart.correct.calculate_attenuation(
        radar,
        0,
        fzl=8000.0,
        gatefilter=None,
        rhv_min=0.5,
        ncp_min=0.1,
        a_coef=0.06,
        beta=0.8,
        refl_field="total_power",
        ncp_field="normalized_coherent_power",
        rhv_field="cross_correlation_ratio",
        phidp_field="differential_phase",
        spec_at_field="specific_attenuation"
    )
    radar.add_field("KDP_corr", spec_at, replace_existing=True)
    radar.add_field("DBTH_corr", cor_z, replace_existing=True)
    return radar

def replace_reflectivity_and_attenuation(ds, corrected_reflectivity, corrected_attenuation):
    # add the corrected variables without replacing the original variables
    dims = ('azimuth', 'range')  # using the correct dimensions: azimuth and range
    ds['DBTH_corr'] = (dims, corrected_reflectivity)
    ds['KDP_corr'] = (dims, corrected_attenuation)

    # add attributes to the new fields indicating they are corrected
    ds['DBTH_corr'].attrs['description'] = 'corrected reflectivity'
    ds['KDP_corr'].attrs['description'] = 'corrected attenuation'

    return ds

def fix_angle(ds):
    ds['time'] = ds.time.load() # Convert time from dask to numpy
    angle_dict = xd.util.extract_angle_parameters(ds)
    start_ang = 0 # Set consistent start/end values
    stop_ang = 360
    angle_res = 1.
    direction = angle_dict["direction"]
    # first find exact duplicates and remove
    ds = xd.util.remove_duplicate_rays(ds)
    # second reindex according to retrieved parameters
    ds = xd.util.reindex_angle(
        ds, start_ang, stop_ang, angle_res, direction, method="nearest"
    )
    ds = ds.expand_dims('volume_time') # Expand for volumes for concatenation
    ds['volume_time'] = [ds.time.min('azimuth').values] # results in 'NaT' values in index. # run
    return ds

def get_dp_moments(fn):
    datasets = []
    ds = xr.open_dataset(fn, engine='iris', group='sweep_0')
    ds0 = fix_angle(ds)
    droplist = ['time', 'VRADH', 'WRADH', 'sweep_mode', 'sweep_number', 'prt_mode', 'follow_mode', 'sweep_fixed_angle']
    ds0_drop = ds0.drop_vars(droplist , errors='ignore')
    # fix volume_time
    #ds0_drop = fix_time_index(ds0_drop, fns[:])
    #ds0_drop = fix_time_index1800(ds0_drop, fns[:]) 
    #ds0_drop = assign_int64time_units_attrs(ds0_drop)
    #ref_datetime = np.datetime64('1800-01-01')
    #da['volume_time'] = ( (da['volume_time'] - ref_datetime) / np.timedelta64(1, 'h'))
    #ds0_drop = add_timezone_to_datetime(ds0_drop)
    #ds0_drop = fillna_dataset(ds0_drop)
    return ds0_drop

def reindex_time_delta_xarray(data, column_name, time_delta):
    # Get the length of the dataset
    length = len(data[column_name])

    # Create a new monotonically increasing datetime index
    start_time = pd.to_datetime(data[column_name].values[0].astype('datetime64[ns]'))
    new_index = pd.date_range(start_time, periods=length, freq=time_delta)

    # Update the 'volume_time' coordinate
    data = data.assign_coords({column_name: new_index})

    return data

def resample_to_hourly(ds):
    ds_hourly = ds.resample(volume_time='1H').mean()
    return ds_hourly

def xarray_fill_nan(da):
    duh = da.fillna(0)
    # check if null
    return duh

def get_radar_objects(fn):
        radar = pyart.io.read_sigmet(fn)
        return radar

def zarr_archive(ds, name):
    path = r'C:\Users\baxter.vieux\SnowQ_ML\notebooks\zarr_archive_TILT3_CORR\{}.zarr'.format(name)
    ds.to_zarr(path, mode='w')
    return path

data_path = r'C:\Users\baxter.vieux\snow\KALA'

for period in period_list:
    start_date, end_date, lat, lon, height = period
    start_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    delta = datetime.timedelta(days=1)
    fns = []
    print(f'Processing period: {start_date} to {end_date}')
    
    # Store the original start date for the current period
    original_start_dt = start_dt
    
    while start_dt <= end_dt:
        ymd = start_dt.strftime('%Y%m%d')
        ym = start_dt.strftime('%Y%m')
        day_path = os.path.join(data_path, ym, ymd)
        day_glob = os.path.join(day_path, '*.*') 
        current_fns = get_fns_start_date_to_end_date(day_glob, start_dt.strftime('%Y%m%d'), end_dt.strftime('%Y%m%d'))
        try:
            files = sorted(glob.glob(day_glob))
        except:
            start_dt += delta
            continue
            
        day_complete = False  # Adding this flag to track if we should move to the next day
        for file in files:
            try:
                current_fns = get_fns_start_date_to_end_date(day_glob, start_dt.strftime('%Y%m%d'), end_dt.strftime('%Y%m%d'))
                if len(current_fns) > 0:
                    fns.extend(current_fns)
                    #print(f' for start_dt: {start_dt} to end_dt: {end_dt} len(fns): {len(fns)}')
                    day_complete = True  # We found files for this day, so we should move to the next day
            except:
                print(f"Error: Incomplete file or missing file: {file}")

            if day_complete:  # If we found files for the day, move to the next day
                break  # Break out of the 'for file in files' loop after processing files for the current day
                
        #print(f'for day {start_dt} current_fns {len(current_fns)} first {current_fns[0]} last {current_fns[-1]}')
        start_dt += delta
        
    if len(fns) > 0:
        _first = datetime.datetime.strptime(os.path.basename(fns[0])[0:6], '%y%m%d').strftime('%Y%m%d')
        _last = datetime.datetime.strptime(os.path.basename(fns[-1])[0:6], '%y%m%d').strftime('%Y%m%d')
        print(f'Converting {len(fns)} RAW files from: {_first} to: {_last}')

        # Initialize an empty list to store xarrays
        ds0_corr_list = []
        for fn in fns:
            try:
                ds0_drop = get_dp_moments(fn)
                ds0_drop = ds0_drop.squeeze()
                radar    = get_radar_objects(fn)

                corr_radar = correct_dual_pol_moments(radar)

                #corrected_reflectivity = corr_radar.fields['corrected_reflectivity_horizontal']['data']
                #corrected_attenuation = corr_radar.fields['corrected_specific_attenuation']['data']
                corrected_reflectivity = corr_radar.fields['DBTH_corr']['data']
                corrected_attenuation = corr_radar.fields['KDP_corr']['data']

                ds0_corr = [
                    replace_reflectivity_and_attenuation(
                        ds0_drop,
                        corrected_reflectivity,
                        corrected_attenuation
                    )
                ]
                ds0_corr_list.append(ds0_corr[0])
            except EOFError:
                print(f"Warning: Unexpected file end or corrupt file detected in {fn}. Skipping this file.")
                continue
            except Exception as e:
                print(f"Error processing file {fn}: {e}")
                continue

        ds0_corr_combined = xr.concat(ds0_corr_list, dim="volume_time")
        ts_delta = pd.to_timedelta('7 minutes 28 seconds')
        ds0_corr_combined = ds0_corr_combined.dropna(dim='volume_time', subset=['volume_time'])
        ds0_corr_combined = reindex_time_delta_xarray(ds0_corr_combined, 'volume_time', ts_delta)

        ds0_corr_combined = reindex_time_delta_xarray(ds0_corr_combined, 'volume_time', ts_delta)
        ds0_corr_combined['DBTH_corr'] = xarray_fill_nan(ds0_corr_combined['DBTH_corr'])
        ds0_corr_combined['KDP_corr'] = xarray_fill_nan(ds0_corr_combined['KDP_corr'])

        ds0_corr_combined_hourly = resample_to_hourly(ds0_corr_combined)
        # Convert DataArray back to Dataset
        ds0_corr_combined_hourly = ds0_corr_combined_hourly

        print(ds0_corr_combined_hourly.info)
        
        # Save the aggregated dataset to zarr
        z_path = zarr_archive(ds0_corr_combined_hourly, f"data_{_first}_{_last}")

        z_store = zarr.open(z_path, mode='r')
        print(f"saved to: {z_path} \n with info: {z_store.info}\n")

        start_dt = original_start_dt
 

NameError: ignored